![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
from datetime import datetime, date
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE

# load and review dataset columns and data types and data points
df = pd.read_csv('rental_info.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [2]:
# convert 'rental_date' and 'return_date' to datetime
df = df.astype({'rental_date': np.datetime64,'return_date': np.datetime64})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rental_date       15861 non-null  datetime64[ns]
 1   return_date       15861 non-null  datetime64[ns]
 2   amount            15861 non-null  float64       
 3   release_year      15861 non-null  float64       
 4   rental_rate       15861 non-null  float64       
 5   length            15861 non-null  float64       
 6   replacement_cost  15861 non-null  float64       
 7   special_features  15861 non-null  object        
 8   NC-17             15861 non-null  int64         
 9   PG                15861 non-null  int64         
 10  PG-13             15861 non-null  int64         
 11  R                 15861 non-null  int64         
 12  amount_2          15861 non-null  float64       
 13  length_2          15861 non-null  float64       
 14  rental_rate_2     1586

In [3]:
# create a column named "rental_length_days" using the columns "return_date" and "rental_date", and add it to the pandas DataFrame
df.rental_length_days = (df.return_date - df.rental_date).dt.days

df.insert(2,'rental_length_days', df.rental_length_days, allow_duplicates=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   rental_date         15861 non-null  datetime64[ns]
 1   return_date         15861 non-null  datetime64[ns]
 2   rental_length_days  15861 non-null  int64         
 3   amount              15861 non-null  float64       
 4   release_year        15861 non-null  float64       
 5   rental_rate         15861 non-null  float64       
 6   length              15861 non-null  float64       
 7   replacement_cost    15861 non-null  float64       
 8   special_features    15861 non-null  object        
 9   NC-17               15861 non-null  int64         
 10  PG                  15861 non-null  int64         
 11  PG-13               15861 non-null  int64         
 12  R                   15861 non-null  int64         
 13  amount_2            15861 non-null  float64   

,rental_date,return_date,rental_length_days,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33,2005-05-28 23:40:33,3,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16,2005-06-18 19:24:16,2,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45,2005-07-17 10:11:45,7,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41,2005-08-02 14:30:41,2,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04,2005-08-23 13:35:04,4,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [4]:
# create two columns of dummy variables from "special_features"
df.deleted_scenes = pd.Series(np.where(df.special_features.str.contains('Deleted Scenes')==True, 1, 0))
df.insert(9,'deleted_scenes', df.deleted_scenes, allow_duplicates=True)

df.behind_the_scenes = pd.Series(np.where(df.special_features.str.contains('Behind the Scenes')==True, 1, 0))
df.insert(10,'behind_the_scenes', df.behind_the_scenes, allow_duplicates=True)
           
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   rental_date         15861 non-null  datetime64[ns]
 1   return_date         15861 non-null  datetime64[ns]
 2   rental_length_days  15861 non-null  int64         
 3   amount              15861 non-null  float64       
 4   release_year        15861 non-null  float64       
 5   rental_rate         15861 non-null  float64       
 6   length              15861 non-null  float64       
 7   replacement_cost    15861 non-null  float64       
 8   special_features    15861 non-null  object        
 9   deleted_scenes      15861 non-null  int64         
 10  behind_the_scenes   15861 non-null  int64         
 11  NC-17               15861 non-null  int64         
 12  PG                  15861 non-null  int64         
 13  PG-13               15861 non-null  int64     

,rental_date,return_date,rental_length_days,amount,release_year,rental_rate,length,replacement_cost,special_features,deleted_scenes,behind_the_scenes,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33,2005-05-28 23:40:33,3,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,1,0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16,2005-06-18 19:24:16,2,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,1,0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45,2005-07-17 10:11:45,7,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,1,0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41,2005-08-02 14:30:41,2,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,1,0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04,2005-08-23 13:35:04,4,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,1,0,0,0,1,8.9401,15876.0,8.9401


In [5]:
# Choose the "rental_length_days" as the target column and save it as a pandas Series called y
X = df.drop(['rental_date', 'return_date', 'rental_length_days', 'special_features'], axis=1)
y = df.rental_length_days

X.info(), y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   amount             15861 non-null  float64
 1   release_year       15861 non-null  float64
 2   rental_rate        15861 non-null  float64
 3   length             15861 non-null  float64
 4   replacement_cost   15861 non-null  float64
 5   deleted_scenes     15861 non-null  int64  
 6   behind_the_scenes  15861 non-null  int64  
 7   NC-17              15861 non-null  int64  
 8   PG                 15861 non-null  int64  
 9   PG-13              15861 non-null  int64  
 10  R                  15861 non-null  int64  
 11  amount_2           15861 non-null  float64
 12  length_2           15861 non-null  float64
 13  rental_rate_2      15861 non-null  float64
dtypes: float64(8), int64(6)
memory usage: 1.7 MB
<class 'pandas.core.series.Series'>
RangeIndex: 15861 entries, 0 to 15860

(None, None)

In [6]:
# Split the data into X_train, y_train, X_test, and y_test train and test set

SEED = 9

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [7]:
lr = LogisticRegression(random_state=SEED)
knr = KNeighborsRegressor()
dtr = DecisionTreeRegressor(random_state=SEED)

In [8]:
classifers = [
    ('Logistic Regression', lr),
    ('K Nearest Neighbors', knr),
    ('Classification Tree', dtr)
]

for clf_name, clf in classifers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("{:s}:{:0.3f}".format(clf_name, MSE(y_test, y_pred)))

Logistic Regression:8.895
K Nearest Neighbors:2.721
Classification Tree:2.162


In [9]:
# Save the model you would recommend as a variable named best_model, and save its MSE on the test set as best_mse
best_model = dtr.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

best_mse = MSE(y_test, y_pred)